In [35]:
# !pip install langchain-teddynote

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("NursingHome")

LangSmith 추적을 시작합니다.
[프로젝트명]
NursingHome


In [3]:
import os
from langchain_upstage import ChatUpstage, UpstageEmbeddings

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """주어진 사용자 질문을 `환자정보`, `메뉴`, 또는 `병원비` 중 하나로 분류하세요. 한 단어 이상으로 응답하지 마세요.

<question>
{question}
</question>

Classification:"""
)
llm = ChatUpstage(api_key=os.getenv("UPSTAGE_API_KEY"))


# 체인을 생성합니다.
chain = (
    prompt
    | llm
    | StrOutputParser()  # 문자열 출력 파서를 사용합니다.
)

생성한 chain을 사용하여 질문을 분류합니다.


In [5]:
# 질문을 입력하여 체인을 호출합니다.
chain.invoke({"question": "아버님 상태는 어떠신가요?"})

'환자정보.'

In [6]:
# 질문을 입력하여 체인을 호출합니다.
chain.invoke({"question": "오늘 아침은 뭐가 나왔어요?"})

'메뉴'

In [7]:
# 질문을 입력하여 체인을 호출합니다.
chain.invoke({"question": "저희 이번달 병원비가 어떻게 되죠?"})

'병원비'

In [8]:
# 질문을 입력하여 체인을 호출합니다.
chain.invoke({"question": "1인실 20일 입원한 환자의 병원비는 얼마정도 예상될까요?"})

'병원비.'

이제 세 개의 하위 체인을 생성해 보겠습니다.


In [9]:
from TEMPLATES.rag_template import prompt, menu_prompt

In [10]:
from rag_menu import menu_chain
from rag import patient_chain
from rag_bill import bill_chain

/Users/seongyeon/Desktop/Nursing-home/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


## 사용자 정의 함수 사용하기

LangChain 공식 도큐먼트에서 권장하는 방식이며, 서로 다른 출력 간의 라우팅을 위해 **사용자 정의 함수**를 `RunnableLambda` 로 래핑하여 활용할 수도 있습니다.




In [11]:
def route(info):
    if "병원비" in info["topic"].lower():
        # datascience_chain을 반환
        return bill_chain
    elif "메뉴" in info["topic"].lower():
        return menu_chain
    # 그 외의 경우
    else:
        # general_chain을 반환
        return patient_chain

In [12]:
from operator import itemgetter
from langchain_core.runnables import RunnableLambda

full_chain = (
    {"topic": chain, "question": itemgetter("question")}
    | RunnableLambda(
        # 경로를 지정하는 함수를 인자로 전달합니다.
        route
    )
    | StrOutputParser()
)

In [13]:
# 수학과 관련된 질문을 입력하여 체인을 호출합니다.
full_chain.invoke({"question": "아버님 상태는 어떠신가요?"})

'환자분의 상태는 좌측 편마비(left hemiplegia)로 인해 침대에 누워 계시는 상태입니다. 또한, 목에 튜브(L-tube, 18Fr)가 삽입되어 있고, 코를 통해 영양 공급이 이루어지고 있습니다. 대소변을 스스로 조절할 수 없어 기저귀를 착용하고 있습니다. 오른쪽 손으로 침대 난간을 반복적으로 치는 행동을 보이고 있습니다. 공기 매트리스를 사용하고 있으며, 몸의 위치를 자주 바꾸어 주어야 합니다.\n\n최근 기록에 따르면, 혈압 96/42, 심박수 59, 호흡 20, 체온 36.3°C로 안정된 상태입니다.\n\n참고한 데이터의 메타데이터:\n- 문서 유형: 간호 기록\n- 문서 날짜: 2023년 10월 2-3일'

In [14]:
# 수학과 관련된 질문을 입력하여 체인을 호출합니다.
full_chain.invoke({"question": "어제 점심은 뭐가 나왔나요?"})

'어제 점심은 쌀밥, 솎음배추된장국, 돈장조림, 고등어구이, 양배추생채, 배추김치가 나왔습니다.\n\n본 식단은 식자재 수급 상황에 따라 다소 변경 될 수 있습니다.'

In [15]:
# 수학과 관련된 질문을 입력하여 체인을 호출합니다.
full_chain.invoke({"question": "1인실 20일 입원한 환자의 병원비는 얼마정도 예상될까요?"})

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


'환자군별 병원비 산출 방식에 따르면, 1인실에서 20일 입원한 환자의 병원비는 다음과 같이 계산됩니다.\n\n1. 공단청구금:\n* 일당정액수가: 85,050원 x 20일 = 1,701,000원\n* 필요인력가산: 51,300원\n* 환자안전관리료: 47,100원\n* 감염관리료: 66,600원\n* 식대: 56,430원 x 20일 = 1,128,600원\n* 총진료비: 1,701,000원 + 51,300원 + 47,100원 + 66,600원 + 1,128,600원 = 2,994,600원\n* 본인부담금: 825,450원\n* 공단청구금: 2,455,350원\n\n2. 별도 청구: 재활, 한방, 혈액투석 등의 별도 청구는 "비보험" 항목에 포함되지 않습니다.\n\n3. 비보험: 간병비 및 병실료차액:\n* 간병비: 1일 약 130,000원 x 20일 = 2,600,000원\n* 병실료차액: 1일 약 70,000원 x 20일 = 1,400,000원\n\n따라서, 1인실에서 20일 입원한 환자의 총 병원비는 다음과 같습니다:\n* 공단청구금: 2,455,350원\n* 본인부담금: 825,450원\n* 간병비: 2,600,000원\n* 병실료차액: 1,400,000원\n\n총 병원비: 2,455,350원 + 825,450원 + 2,600,000원 + 1,400,000원 = 7,280,800원'